<a href="https://colab.research.google.com/github/mahmutakar0/Bk/blob/main/bk001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Packages

Gerekli paketlerin yüklenmesi

In [315]:
# BASE (Temel)
# ------------------------------------------------------
import numpy as np
import pandas as pd
import os
import gc
import warnings

# PACF(kısmi otokorelasyon) - ACF(otokorelasyon fonksiyonu)
# ------------------------------------------------------
import statsmodels.api as sm

# DATA VISUALIZATION (Veri görselleştirme)
# ------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# CONFIGURATIONS (Yapılandırmalar)
# ------------------------------------------------------
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.2f}'.format
warnings.filterwarnings('ignore')

# Metrics & Evaluation (Ölçütler ve değerlendirme)
# -----------------------------------
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_squared_log_error
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV

# 2. İmporting

Verinin yüklenmesi

In [316]:
# Import
# ------------------------------------------------------
url = "https://raw.githubusercontent.com/mahmutakar0/Bk/main/databek.csv"
data = pd.read_csv(url,sep=";")
satisdata = data.copy()

# Renaming of features (Özelliklerin yeniden adlandırılması)
# ------------------------------------------------------
data = data.rename(columns={"HAFTA": "hafta", "STOK_ADI": "stokadı",  "TARIH": "tarih", "MUSTERI_ISMI": "musteri",
                            "MIKTAR3": "kg", "USD/KG": "usd/kg","KUR": "kur", "DOV_TUTAR": "tutar", "KOD_5": "ürün",
                            "KOD_4": "özürün","SATIS_TEMSILCISI": "s_temsilcisi", "CARI_ACILIS": "c_acilis",
                            "SEHİR": "sehir","ULKE_KODU": "ülke", "ISLEM_YERI": "yer", "DAHIL": "dahil",
                            "CARI_GRUP_KODU":"bölge", "ISLEM_TURU": "islem", "UNVAN":"ünvan", "GRUP_KODU": "ürüngurupkodu"
                            })
data["ürün"] = data["ürün"].replace({
    "TD - 3.T.DİZ.ÇİVİLER": "Td. çiviler (TD)",
                                     "DC - 1.DÜZ ÇİVİLER": "Düz çiviler (DC)",
                                     "OC - 2.OVALI ÇİVİLER": "Ovalı çiviler (OC)",
                                     "TL - 4.TELLER": "Teller (TL)",
                                     "PS - 5.PLASTİĞE DİZİLİ ÇİVİLER": "Plastiğe dizili çiviler (PS)"
                                     })
data["özürün"] = data["özürün"].replace({"RS - 07.RİNG TELE DİZİLİ": "Ring tele dizili (RS)",
                                         "TC - 10.TEL ÇİVİ": "Tel çivi (TC)",
                                         "EP - 05.EPAL DÖKME": "Epal dökme (EP)",
                                         "DS - 06.DÜZ TELE DİZİLİ": "Düz tele dizili (DS)",
                                         "HS - 08.HELİKAL TELE DİZİLİ": "Helikal tele dizili (HS)",
                                         "IN - 01.İNŞAAT ÇİVİSİ": "İnşaat çivisi (IN)",
                                         "RD - 03.RİNG DÖKME": "Ring Dökme (RD)",
                                         "DD - 02.DÜZ DÖKME": "Düz Dökme (DD)",
                                         "TT - 11.TAVLI TEL": "Tavlı tel (TT)",
                                         "PD - 14.DÜZ PLASTİĞE DİZİLİ": "Düz plastik dizili (PD)",
                                         "TDEP - 09.EPAL TELE DİZİLİ": "Epal tele dizili (TDEP)",
                                         "HD - 04.HELİKAL DÖKME": "Helikal dökme (HD)",
                                         "PH - 16.HELİKAL PLASTİĞE DİZİLİ": "Helikal plastik dizili (PH)",
                                         "ST - 12.SERT TEL": "Sert tel (ST)",
                                         "DT - 13.DÜZ TEL": "Düz tel (DT)",
                                         "PR - 15.RİNG PLASTİĞE DİZİLİ": "Ring plastik dizili (PR)"
                                         })
# Editing data that is null (Null olan verinin düzenlenmesi)
# ------------------------------------------------------
data.dropna(subset=["tutar"],inplace=True)
data["kg"] = data["kg"].str.replace(",",".").astype(float)
data = data[data["kg"] != 0]
data = data.fillna(0)


# Editing of datatypes (Veri tiplerinin düzenlenmesi)
# ------------------------------------------------------

data["usd/kg"] = data["usd/kg"].str.replace(",",".").astype(float)
data["tutar"] = data["tutar"].str.replace(",",".").astype(float)
data["kur"] = data["kur"].str.replace(",", ".").astype(float)
data["bölge"] = data["bölge"].astype(int)
data["tarih"] = pd.to_datetime(data.tarih, format='%d.%m.%Y')

data.head(2)

,hafta,stokadı,musteri,tarih,kg,usd/kg,tutar,kur,islem,ünvan,ürüngurupkodu,özürün,ürün,s_temsilcisi,sehir,ülke,c_acilis,bölge,dahil,yer
0,41,"T.DIZ.ÇIVI-RING_2,50x055x6,1mm 30x300 ELM DU_",EMİN PALET AMBALAJ ORMAN ÜRÜNLERİ SAN.TİC.LTD....,2023-10-12,992.63,0.88,875.79,27.69,SATIS,BEKSAN,3 - MAMUL,Ring tele dizili (RS),Td. çiviler (TD),TARIK DARICI,İZMİR,TR,17.08.2023,2,DIGER,Y.IÇI
1,41,"T.DIZ.ÇIVI-RING_2,50x040x6,1mm 40x300 ELM DU_",ALPAGUT ENTEGRE ATIK YÖN. SAN. VE TİC. LTD. ŞTİ,2023-10-12,96.75,1.09,105.82,27.69,SATIS,BEKSAN,3 - MAMUL,Ring tele dizili (RS),Td. çiviler (TD),BÜŞRA PAYLAN,BALIKESİR,TR,2.02.2022,5,DIGER,Y.IÇI


In [317]:
# Time-based groupings of data (Verilerin zaman bazlı guruplamaları)
# ------------------------------------------------------
satisdatakg = data.sort_values("kg",ascending=False)
satisdatakg.reset_index(inplace=True)
satisdatakg.drop("index", axis=1,inplace=True)
satisdatakg.drop(0,axis=0,inplace=True)
satisdatakg.sort_values("tarih", ascending=True, inplace=True)

satisdatad = data.sort_values("kg",ascending=False)
satisdatad.reset_index(inplace=True)
satisdatad.drop("index", axis=1,inplace=True)
satisdatad.drop(0,axis=0,inplace=True)
satisdatad.sort_values("tarih", ascending=True, inplace=True)

a = satisdatakg.set_index("tarih").groupby("ürün").resample("D").kg.sum().reset_index()
b = satisdatad.set_index("tarih").groupby("ürün").resample("D").tutar.sum().reset_index()
c = satisdatakg.set_index("tarih").groupby("özürün").resample("D").kg.sum().reset_index()
d = satisdatad.set_index("tarih").groupby("özürün").resample("D").tutar.sum().reset_index()


# visualization of data (Verilerin Görselleştirilmesi)
# ------------------------------------------------------
fig0 = px.line(a.sort_values("tarih"),
       x="tarih", y="kg",
       color="ürün",
       title="Ürünlerin günlük satış toplamları (kg)"
       )

fig1 = px.line(b.sort_values("tarih"),
       x="tarih", y="tutar",
       color="ürün",
       title="Ürünlerin günlük satış toplamları ($)")

fig2 = px.line(c.sort_values("tarih"),
       x="tarih", y="kg",
       color="özürün",
       title="Özürünlerin günlük toplam satışları (kg)")

fig3 = px.line(d.sort_values("tarih"),
       x="tarih", y="tutar",
       color="özürün",
       title="Özürünlerin günlük toplam satışları ($)")

fig0.update_layout(title_x=0.5)
fig1.update_layout(title_x=0.5)
fig2.update_layout(title_x=0.5)
fig3.update_layout(title_x=0.5)

fig0.show()
fig1.show()
fig2.show()
fig3.show()